# Hiding arguments from the model

- InjectedArgs 를 이용하여 모델이 제어해서는 안되는 인수를 숨긴다.

In [2]:
from typing import List

from langchain_core.tools import InjectedToolArg, tool
from typing_extensions import Annotated

user_to_pets = {}

@tool(parse_docstring=True) # 함수의 docstring 을 읽음
def update_favorite_pets(
    pets: List[str], user_id: Annotated[str, InjectedToolArg]
) -> None:
    """Add the list of favorite pets.
    
    Args:
        pets: List of favorite pets to set.
        user_id: User's ID
    """
    user_to_pets[user_id] = pets

@tool(parse_docstring=True)
def delete_favorite_pets(user_id: Annotated[str, InjectedToolArg]) -> None:
    """Delete the list of favorite pets.

    Args:
        user_id: User's ID.
    """
    if user_id in user_to_pets:
        del user_to_pets[user_id]


@tool(parse_docstring=True)
def list_favorite_pets(user_id: Annotated[str, InjectedToolArg]) -> None:
    """List favorite pets if any.

    Args:
        user_id: User's ID.
    """
    return user_to_pets.get(user_id, [])

## get_input_schema()

- 도구의 전체 입력 스키마를 반환한다.
- 즉, 함수가 필요로 하는 모든 입력 매개변수(인자)들이 정의되어 있음
- 이는 직접 호출할때 (`invoke()` 사용) 필요한 입력을 보여주는 것

In [3]:
update_favorite_pets.get_input_schema().model_json_schema()

## input schemas 를 살펴보면 `user_id` 는 여전히 존재

{'description': 'Add the list of favorite pets.',
 'properties': {'pets': {'description': 'List of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'},
  'user_id': {'description': "User's ID",
   'title': 'User Id',
   'type': 'string'}},
 'required': ['pets', 'user_id'],
 'title': 'update_favorite_pets',
 'type': 'object'}

## tool_call_schema()

- 모델이 도구를 호출할 떄 사용할 스키마를 정의한다.
- LLM 이 자동으로 도구를 사용할 때, 어떤 매개변수를 생성해야 하는지를 결정한다.


In [4]:
update_favorite_pets.tool_call_schema.model_json_schema()

{'description': 'Add the list of favorite pets.',
 'properties': {'pets': {'description': 'List of favorite pets to set.',
   'items': {'type': 'string'},
   'title': 'Pets',
   'type': 'array'}},
 'required': ['pets'],
 'title': 'update_favorite_pets',
 'type': 'object'}

In [5]:
user_id = "123"
update_favorite_pets.invoke({"pets": ["lizard", "dog"], "user_id": user_id})
print(user_to_pets)
print(list_favorite_pets.invoke({"user_id": user_id}))

{'123': ['lizard', 'dog']}
['lizard', 'dog']


## Model Load

- Ollama 를 이용해서 `Qwen2.5:7b` 를 사용하도록 하겠다.

In [22]:
from langchain_openai import ChatOpenAI

model = 'qwen2.5:7b'
key = 'ollama'

llm = ChatOpenAI(
    base_url = "http://127.0.0.1:11434/v1",
    api_key=key,
    model=model
)

In [24]:
tools = [
    update_favorite_pets,
    delete_favorite_pets,
    list_favorite_pets,
]
llm_with_tools = llm.bind_tools(tools)
ai_msg = llm_with_tools.invoke("my favorite animals are cats and parrots")
ai_msg.tool_calls

[{'name': 'update_favorite_pets',
  'args': {'pets': ['cats', 'parrots']},
  'id': 'call_c1gd70ab',
  'type': 'tool_call'}]